#  Tokamak Benchmark (no toroidal launch angle)

In [ ]:
# constructing the data needed -- magnetic field data, ne data, polflux data

import json
import numpy as np
import scipy.constants as const
from scotty.fun_general import find_q_lab
from scipy.interpolate import RectBivariateSpline

# Data location for B field and ne/polflux
topfile_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\topfile_189998_3000ms_quinn.json"
nefile_path  = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\ne_189998_3000ms_quinn.dat"

with open(topfile_path, "r") as topfile:
    topfile_data = json.load(topfile)

R_coordinates  = topfile_data.get("R", [])
Z_coordinates  = topfile_data.get("Z", [])
Br_field = topfile_data.get("Br", [])
Bt_field = topfile_data.get("Bt", [])
Bz_field = topfile_data.get("Bz", [])
polflux_field = topfile_data.get("pol_flux", [])

num_R_points = len(R_coordinates)  # R has 129 data points in total
min_R = min(R_coordinates) # 0.84
max_R = max(R_coordinates) # 2.54

num_Z_points = len(Z_coordinates)  # Z has 129 data points in total also
min_Z = min(Z_coordinates) # -1.6
max_Z = max(Z_coordinates) # 1.6

# This means X and Z should now have 129 data points.
# The Y (toroidal direction) should have just enough points for interpolation.
# So maybe 20 points or so?
X = np.linspace(start=min_R, stop=max_R, num=num_R_points+1)
Y = np.linspace(start=-0.1,  stop=0.1,   num=21)
Z = np.linspace(start=min_Z, stop=max_Z, num=num_Z_points+1)

# Creating the interpolators
Br_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, Br_field, kx=5, ky=5)
Bt_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, Bt_field, kx=5, ky=5)
Bz_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, Bz_field, kx=5, ky=5)
polflux_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, polflux_field, kx=5, ky=5)

# Calculate the value of Bx, By, Bz, polflux at a particular coordinate
def calculate_values(X_coord, Y_coord, Z_coord):
    R_coord = np.sqrt(X_coord**2 + Y_coord**2)
    zeta_coord = np.arctan(Y_coord / X_coord)

    Br_value = Br_field_spline_class(R_coord, Z_coord)
    Bt_value = Bt_field_spline_class(R_coord, Z_coord)
    Bz_value = Bz_field_spline_class(R_coord, Z_coord)
    polflux_value = polflux_field_spline_class(R_coord, Z_coord)

    Bx_value = (Br_value*X_coord - Bt_value*Y_coord) / R_coord
    By_value = (Br_value*Y_coord + Bt_value*X_coord) / R_coord

    return Bx_value, By_value, Bz_value, polflux_value

# Initialise empty arrays to put data into
zero_array = np.zeros((len(Z), len(Y), len(X)))
Bx_array = np.copy(zero_array)
By_array = np.copy(zero_array)
Bz_array = np.copy(zero_array)
polflux_array = np.copy(zero_array)

for i in range(len(X)):
    for j in range(len(Y)):
        for k in range(len(Z)):
            X_coord = X[i]
            Y_coord = Y[j]
            Z_coord = Z[k]
            Bx_value, By_value, Bz_value, polflux_value = calculate_values(X_coord, Y_coord, Z_coord)

            Bx_array[k][j][i] = float(Bx_value)
            By_array[k][j][i] = float(By_value)
            Bz_array[k][j][i] = float(Bz_value)
            polflux_array[k][j][i] = float(polflux_value)





# Writing the data into files that 3D Scotty can read
topfile_data = {
    "X": X.tolist(),
    "Y": Y.tolist(),
    "Z": Z.tolist(),
    "Bx": Bx_array.tolist(),
    "By": By_array.tolist(),
    "Bz": Bz_array.tolist(),
    "pol_flux": polflux_array.tolist()
}

with open("topfile_for3dscotty.json", "w") as topfile:
    json.dump(topfile_data, topfile, indent=4)

In [ ]:
# Checking if the data that was just generated is correct and that it is read correctly
# Create another spline to read the data that was just written

import numpy as np
import pathlib
import scipy.constants as const

from scotty.beam_me_up_3D_temp import create_magnetic_geometry_3D
scotty_3d_topfile_path_without_filename = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"
field_3d_scotty = create_magnetic_geometry_3D(
    find_B_method = "omfit_3D",
    magnetic_data_path = pathlib.Path(scotty_3d_topfile_path_without_filename),
    input_filename_suffix = "",
    interp_order = 5
)

from scotty.beam_me_up import create_magnetic_geometry
scotty_2d_topfile_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark"
field_2d_scotty = create_magnetic_geometry(
    find_B_method = "omfit",
    magnetic_data_path = pathlib.Path(scotty_2d_topfile_path),
    input_filename_suffix = "_189998_3000ms_quinn",
    interp_order = 5
)

rand_R = np.random.uniform(0.84, 2.56)
rand_zeta = np.random.uniform(0, 2*const.pi)

rand_X = rand_R*np.cos(rand_zeta)
rand_Y = rand_R*np.sin(rand_zeta)
rand_Z = np.random.uniform(-1.6, 1.6)

print(field_3d_scotty.B_X(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.B_R(rand_R, rand_Z))
print()
print(field_3d_scotty.B_Y(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.B_T(rand_R, rand_Z))
print()
print(field_3d_scotty.B_Z(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.B_Z(rand_R, rand_Z))
print()
print(field_3d_scotty.polflux(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.poloidal_flux(rand_R, rand_Z))

In [ ]:
# the actual beam tracing code

import numpy as np
import os
import pathlib
from scotty.beam_me_up_3D_temp import beam_me_up_3D

kwargs_dict = {
    'poloidal_launch_angle_Torbeam': -7.0,
    'toroidal_launch_angle_Torbeam': 0.0,
    'launch_freq_GHz': 72.5,
    'mode_flag': -1,
    'launch_beam_width': 0.1265,
    'launch_beam_curvature': -0.95,
    'find_B_method': "omfit_3D",
    'Psi_BC_flag': 'discontinuous', # NOT USED FOR 3D, BUT PUT HERE FOR REFERENCE
    'poloidal_flux_enter': 0.95,
    'poloidal_flux_zero_density': 1.0,
    'figure_flag': True, # NOT USED FOR 3D, BUT PUT HERE FOR REFERENCE
    'vacuum_propagation_flag': True, # NOT USED FOR 3D, BUT PUT HERE FOR REFERENCE
    'vacuumLaunch_flag': False,       # actual value is True, but False used for my 3D Scotty
    'ne_data_path': pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"),
    'magnetic_data_path': pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"),
    'input_filename_suffix': "",
    'output_path': pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"),
}

kwargs_dict["delta_X"] = -1e-5
kwargs_dict["delta_Y"] = -1e-5
kwargs_dict["delta_Z"] = -1e-5
kwargs_dict["delta_K_X"] = 1e-5
kwargs_dict["delta_K_Y"] = 1e-5
kwargs_dict["delta_K_Z"] = 1e-5
kwargs_dict["interp_smoothing"] = 0
kwargs_dict["interp_order"] = 5
kwargs_dict["len_tau"] = 1002
kwargs_dict["rtol"] = 1e-4
kwargs_dict["atol"] = 1e-7





if kwargs_dict['poloidal_launch_angle_Torbeam'] == 0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 42 # not exact
    kwargs_dict["launch_position_cartesian"] = np.array([2.2621936543069086, -0.00040362814170096634, -0.09163904444906242])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-900.6297175486496, 0.16069335998831755, -50.543485174031446])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
 [[-3.46342996e+03   +5.53335966j,     -2.28386497e+00  -19.98312296j,      4.33645205e+02  -88.91438058j],
  [-2.28386497e+00  -19.98312296j,     -4.92365259e+03+1851.21452832j,      1.40497528e+03 -146.34738761j],
  [ 4.33645205e+02  -88.91438058j,      1.40497528e+03 -146.34738761j,     -3.88631484e+03+1551.13527391j]]
)

elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -7.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 40.031518240219356
    kwargs_dict["launch_position_cartesian"] = np.array([2.26436572068288, 0.0009066122128788577, 0.005151249838071285])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-879.5700312013402, -0.3521643721619699, 228.99532936732646])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-3855.45812939 +115.51896532j,      41.6785787   +53.18414012j,      -1255.90393722 +425.70495113j],
 [   41.6785787   +53.18414012j,     -4932.84061887+1881.21400333j,     801.28595773  -65.63402721j ],
 [-1255.90393722 +425.70495113j,      801.28595773  -65.63402721j,     -3958.98499238+1605.52821159j]]
)

elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -10.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 37.11475407679377
    kwargs_dict["launch_position_cartesian"] = np.array([2.263427097581486, 0.0013114878516376572, 0.04728357416749444])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-846.3452459490945, -0.49039419451130006, 355.618568463616])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4611.46076077 +329.02434708j,      716.67508628   +3.04269807j,      -2247.71067532 +763.45958508j],
 [  716.67508628   +3.04269807j,     -5125.6015488 +1941.2629018j,       2510.80973791 -271.37972463j],
 [-2247.71067532 +763.45958508j,      2510.80973791 -271.37972463j,     -4529.76072795+1811.21688397j]]
)

elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -14.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 30.613455225966494
    kwargs_dict["launch_position_cartesian"] = np.array([2.2598765012509063, 0.0014453779662245668, 0.10396136575632378])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-780.9060774194234, -0.49945403537234, 526.2866721978481])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-12826.48959136+1032.60551938j,      2735.24430304 -161.67060233j,     -5353.52552418+1524.56699891j],
 [  2735.24430304 -161.67060233j,     -5663.29793949+2100.58972225j,      4867.72711371 -523.73792424j],
 [ -5353.52552418+1524.56699891j,      4867.72711371 -523.73792424j,     -5448.62479053+2289.76387434j]]
)

elif kwargs_dict['poloidal_launch_angle_Torbeam'] == 0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 5.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 48.00885946028994
    kwargs_dict["launch_position_cartesian"] = np.array([2.2573739496260554, -0.07244943850334908, -0.09257268650012045])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-855.606915560603, 27.460333110611444, -52.17826188610897])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4240.76190939 +206.20818307j,     1487.95189791 -633.3132573j,       106.2658659   -96.50255403j],
 [ 1487.95189791 -633.3132573j,     -5148.30666812+2064.93410939j,      1422.98399696 -142.32152851j],
 [  106.2658659   -96.50255403j,     1422.98399696 -142.32152851j,     -3892.78016806+1647.79533968j]]
)

elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -7.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 3.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 40 # not exact
    kwargs_dict["launch_position_cartesian"] = np.array([2.264231880717698, -0.04126152969955705, 0.00498947454032394])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-873.7910411730103, 15.923260909628633, 229.7898776318594])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4009.83727723 +148.41930689j,      859.89033077 -286.22473174j,     -1386.54610295 +426.66703997j],
 [  859.89033077 -286.22473174j,     -4971.41683884+1909.4825387j,      847.07474049  -92.38987415j ],
 [-1386.54610295 +426.66703997j,      847.07474049  -92.38987415j,     -3994.94497652+1619.50324177j]]
)

elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -7.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == -3.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 40 # not exact
    kwargs_dict["launch_position_cartesian"] = np.array([2.262978175394322, 0.04328914203565364, 0.005996410553292083])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-861.7230027385247, -16.484140177108813, 229.65844167041607])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4114.79346119 +214.93951884j,     -862.97953786 +399.88416019j,     -1177.05256775 +456.39483165j],
 [ -862.97953786 +399.88416019j,     -4987.60529283+1938.02388273j,     791.79271543  -46.37871134j ],
 [-1177.05256775 +456.39483165j,      791.79271543  -46.37871134j,     -3957.86451428+1640.40048732j]]
)

else:
    print("Need to find initial conditions from 2D Scotty first!")
    exit()





output = beam_me_up_3D(**kwargs_dict)

In [ ]:
# data generated by 3d Scotty is returned point-wise
# we convert it to variable-wise (i.e. an array of X coords, K_x values, etc) for plotting
import matplotlib.pyplot as plt
scotty_3d_data = {
    "X_coords": [],
    "Y_coords": [],
    "Z_coords": [],
    "K_X_values": [],
    "K_Y_values": [],
    "K_Z_values": [],
    "Psi_3D_cartesian": [],
    "tau_values_unadjusted": [],
    "dH_dX": [],
    "dH_dY": [],
    "dH_dZ": [],
    "dH_dKx": [],
    "dH_dKy": [],
    "dH_dKz": [],
    "d2H_dX2": [],
    "d2H_dY2": [],
    "d2H_dZ2": [],
    "d2H_dX_dY": [],
    "d2H_dX_dZ": [],
    "d2H_dY_dZ": [],
    "d2H_dKx2": [],
    "d2H_dKy2": [],
    "d2H_dKz2": [],
    "d2H_dKx_dKy": [],
    "d2H_dKx_dKz": [],
    "d2H_dKy_dKz": [],
    "d2H_dX_dKx": [],
    "d2H_dX_dKy": [],
    "d2H_dX_dKz": [],
    "d2H_dY_dKx": [],
    "d2H_dY_dKy": [],
    "d2H_dY_dKz": [],
    "d2H_dZ_dKx": [],
    "d2H_dZ_dKy": [],
    "d2H_dZ_dKz": [],
}
for point in range(len(output)):
    for i, key in enumerate(scotty_3d_data.keys()):
        scotty_3d_data[key].append(output[point][i])

# we need to adjust the tau values for the 3d output (it starts at '0')
# the tau values for the 2d output start at 40 something because we launch the ray from slightly deeper in the plasma due to poor derivative calculations on the outside (which correspond to small and large tau values)
scotty_3d_data["tau_values"] = np.array(scotty_3d_data["tau_values_unadjusted"]) + tau_adjustment # this value corresponds to the "first" corresponding tau for 2d Scotty

# now we find the elements of Psi individually
scotty_3d_data["Psi_3D_cartesian_XX_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][0][0]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XY_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][0][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XZ_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][0][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YY_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][1][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YZ_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][1][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_ZZ_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][2][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XX_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][0][0]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XY_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][0][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XZ_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][0][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YY_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][1][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YZ_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][1][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_ZZ_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][2][2]) for i in range(len(scotty_3d_data["tau_values"]))]










# data generated (B field, ne, polflux) by 2d Scotty is in the h5 file, and in cylindrical
# the lists below are sliced starting from index 75 because the derivatives at the start are quite ugly
# so, in 3d Scotty, we elect to launch the wave from a start point which corresponds to the 75th tau (arbitrary)
# that is produced by 2d Scotty
scotty_2d_data = {}
h5file_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\scotty_output.h5"
import h5py
from scotty.fun_general import find_Psi_3D_lab_Cartesian
with h5py.File(h5file_path, "r") as h5file:
    print(h5file["solver_output"].keys())
    scotty_2d_data["tau_values"] = list(h5file["solver_output"]["tau"][75:])
    print("Value of tau at the 75th index:", scotty_2d_data["tau_values"][0])

    scotty_2d_data["R_coords"] = np.array(list(h5file["solver_output"]["q_R"][75:]))
    scotty_2d_data["zeta_coords"] = np.array(list(h5file["solver_output"]["q_zeta"][75:]))
    scotty_2d_data["X_coords"] = scotty_2d_data["R_coords"]*np.cos(scotty_2d_data["zeta_coords"])
    scotty_2d_data["Y_coords"] = scotty_2d_data["R_coords"]*np.sin(scotty_2d_data["zeta_coords"])
    scotty_2d_data["Z_coords"] = list(h5file["solver_output"]["q_Z"][75:])

    scotty_2d_data["K_R_values"] = list(h5file["solver_output"]["K_R"][75:])
    scotty_2d_data["K_Z_values"] = list(h5file["solver_output"]["K_Z"][75:])

    # scotty_2d_data["Psi_3D_cylindrical"] = list(h5file["solver_output"]["Psi_3D"][75:]) # REMEMBER to un-comment find_Psi_3D_lab_Cartesian below
    scotty_2d_data["Psi_3D_cartesian"] =  list(h5file["analysis"]["Psi_3D_Cartesian"][75:])

    # Converting cylindrical derivatives to cartesian
    scotty_2d_data["dH_dR"] = np.array((h5file["analysis"]["dH_dR"][75:]))
    scotty_2d_data["dH_dZ"] = np.array((h5file["analysis"]["dH_dZ"][75:]))
    scotty_2d_data["dH_dKR"] = np.array((h5file["analysis"]["dH_dKR"][75:]))
    scotty_2d_data["dH_dKzeta"] = np.array((h5file["analysis"]["dH_dKzeta"][75:]))
    scotty_2d_data["dH_dKz"] = np.array((h5file["analysis"]["dH_dKZ"][75:]))

    scotty_2d_data["dH_dX"] = np.array(scotty_2d_data["dH_dR"])*np.array(scotty_2d_data["X_coords"])/np.array(scotty_2d_data["R_coords"])
    scotty_2d_data["dH_dY"] = np.array(scotty_2d_data["dH_dR"])*np.array(scotty_2d_data["Y_coords"])/np.array(scotty_2d_data["R_coords"])
    scotty_2d_data["dH_dKx"] = np.array(scotty_2d_data["dH_dKR"])*np.array(scotty_2d_data["X_coords"])/np.array(scotty_2d_data["R_coords"]) - np.array(scotty_2d_data["dH_dKzeta"])*np.array(scotty_2d_data["Y_coords"])/np.array(scotty_2d_data["R_coords"]**2)
    scotty_2d_data["dH_dKy"] = np.array(scotty_2d_data["dH_dKR"])*np.array(scotty_2d_data["Y_coords"])/np.array(scotty_2d_data["R_coords"]) + np.array(scotty_2d_data["dH_dKzeta"])*np.array(scotty_2d_data["X_coords"])/np.array(scotty_2d_data["R_coords"]**2)

# converting Psi_3D from cylindrical to cartesian (both in lab frame) and then storing the elements individually
scotty_2d_data["Psi_3D_cartesian_XX_Re"] = []
scotty_2d_data["Psi_3D_cartesian_XY_Re"] = []
scotty_2d_data["Psi_3D_cartesian_XZ_Re"] = []
scotty_2d_data["Psi_3D_cartesian_YY_Re"] = []
scotty_2d_data["Psi_3D_cartesian_YZ_Re"] = []
scotty_2d_data["Psi_3D_cartesian_ZZ_Re"] = []
scotty_2d_data["Psi_3D_cartesian_XX_Im"] = []
scotty_2d_data["Psi_3D_cartesian_XY_Im"] = []
scotty_2d_data["Psi_3D_cartesian_XZ_Im"] = []
scotty_2d_data["Psi_3D_cartesian_YY_Im"] = []
scotty_2d_data["Psi_3D_cartesian_YZ_Im"] = []
scotty_2d_data["Psi_3D_cartesian_ZZ_Im"] = []

for i in range(len(scotty_2d_data["tau_values"])):
    Psi_3D_to_store = scotty_2d_data["Psi_3D_cartesian"][i] # find_Psi_3D_lab_Cartesian(scotty_2d_data["Psi_3D_cylindrical"][i],
                                                            #                           scotty_2d_data["R_coords"][i],
                                                            #                           scotty_2d_data["zeta_coords"][i],
                                                            #                           scotty_2d_data["K_R_values"][i],
                                                            #                           0) # K_zeta is 0?
    scotty_2d_data["Psi_3D_cartesian_XX_Re"].append(np.real(Psi_3D_to_store[0][0]))
    scotty_2d_data["Psi_3D_cartesian_XY_Re"].append(np.real(Psi_3D_to_store[0][1]))
    scotty_2d_data["Psi_3D_cartesian_XZ_Re"].append(np.real(Psi_3D_to_store[0][2]))
    scotty_2d_data["Psi_3D_cartesian_YY_Re"].append(np.real(Psi_3D_to_store[1][1]))
    scotty_2d_data["Psi_3D_cartesian_YZ_Re"].append(np.real(Psi_3D_to_store[1][2]))
    scotty_2d_data["Psi_3D_cartesian_ZZ_Re"].append(np.real(Psi_3D_to_store[2][2]))
    scotty_2d_data["Psi_3D_cartesian_XX_Im"].append(np.imag(Psi_3D_to_store[0][0]))
    scotty_2d_data["Psi_3D_cartesian_XY_Im"].append(np.imag(Psi_3D_to_store[0][1]))
    scotty_2d_data["Psi_3D_cartesian_XZ_Im"].append(np.imag(Psi_3D_to_store[0][2]))
    scotty_2d_data["Psi_3D_cartesian_YY_Im"].append(np.imag(Psi_3D_to_store[1][1]))
    scotty_2d_data["Psi_3D_cartesian_YZ_Im"].append(np.imag(Psi_3D_to_store[1][2]))
    scotty_2d_data["Psi_3D_cartesian_ZZ_Im"].append(np.imag(Psi_3D_to_store[2][2]))

from scotty.fun_general import find_K_lab_Cartesian
scotty_2d_data["K_X_values"] = []
scotty_2d_data["K_Y_values"] = []
for i in range(len(scotty_2d_data["R_coords"])):
    K_X_append, K_Y_append, _ = find_K_lab_Cartesian((scotty_2d_data["K_R_values"][i], 0, scotty_2d_data["K_Z_values"][i]),
                                                              (scotty_2d_data["R_coords"][i], scotty_2d_data["zeta_coords"][i], scotty_2d_data["Z_coords"][i]))
    scotty_2d_data["K_X_values"].append(K_X_append)
    scotty_2d_data["K_Y_values"].append(K_Y_append)










# just to see some data
# print(scotty_3d_data["K_Y_values"])
# print(scotty_2d_data["zeta_coords"])
# print(scotty_3d_data["tau_values"])
# print(scotty_2d_data["tau_values"])










import matplotlib.pyplot as plt
""" RAY TRAJECTORY """
# plot both together on a 3d plot
fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.plot(scotty_3d_data["X_coords"], scotty_3d_data["Y_coords"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Trajectory") # numerical
ax.plot(scotty_2d_data["X_coords"], scotty_2d_data["Y_coords"], scotty_2d_data["Z_coords"], color='red', linestyle='-', linewidth=2, zorder=1, label="2d Scotty Trajectory") # numerical
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
ax.set_title("Ray Trajectory")
ax.legend()
#ax.set_ylim(-0.1,0.1)
plt.show()

# plot both together on a 2d plot
plt.plot(scotty_3d_data["X_coords"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Trajectory") # numerical
plt.plot(scotty_2d_data["X_coords"], scotty_2d_data["Z_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty Trajectory") # numerical
plt.xlabel("X-axis")
plt.ylabel("Z-axis")
plt.title("Ray Trajectory")
plt.legend()
plt.show()



""" POSITION """
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
plt.tight_layout()
axs[0].plot(scotty_3d_data["tau_values"], scotty_3d_data["X_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty X coord") # numerical
axs[0].plot(scotty_2d_data["tau_values"], scotty_2d_data["X_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty X coord") # numerical
axs[0].set_title("X coord vs tau")
axs[0].legend(loc="best")
axs[0].grid(True, which="both")
axs[1].plot(scotty_3d_data["tau_values"], scotty_3d_data["Y_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Y coord") # numerical
axs[1].plot(scotty_2d_data["tau_values"], scotty_2d_data["Y_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty Y coord") # numerical
axs[1].set_title("Y coord vs tau")
axs[1].legend(loc="best")
axs[1].grid(True, which="both")
axs[2].plot(scotty_3d_data["tau_values"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Z coord") # numerical
axs[2].plot(scotty_2d_data["tau_values"], scotty_2d_data["Z_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty Z coord") # numerical
axs[2].set_title("Z coord vs tau")
axs[2].legend(loc="best")
axs[2].grid(True, which="both")
plt.show()



""" WAVEVECTORS """
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
plt.tight_layout()
axs[0].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_X_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty K_X value") # numerical
axs[0].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_X_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty K_X value") # numerical
axs[0].set_title("K_X vs tau")
axs[0].legend(loc="best")
axs[0].grid(True, which="both")
axs[1].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_Y_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty K_Y value") # numerical
axs[1].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_Y_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty K_Y value") # numerical
axs[1].set_title("K_Y vs tau")
axs[1].legend(loc="best")
axs[1].grid(True, which="both")
axs[2].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_Z_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty K_Z value") # numerical
axs[2].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_Z_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty K_Z value") # numerical
axs[2].set_title("K_Z vs tau")
axs[2].legend(loc="best")
axs[2].grid(True, which="both")
plt.show()



""" PSI ELEMENTS """
fig, axs = plt.subplots(4, 3, figsize=(20, 15))
axs = axs.flatten()
plt.tight_layout()
counter_Re = 0
counter_Im = 3
for element, element_key in [("Psi_XX", "Psi_3D_cartesian_XX"), ("Psi_XY", "Psi_3D_cartesian_XY"), ("Psi_XZ", "Psi_3D_cartesian_XZ"), ("Psi_YY", "Psi_3D_cartesian_YY"), ("Psi_YZ", "Psi_3D_cartesian_YZ"), ("Psi_ZZ", "Psi_3D_cartesian_ZZ")]:
    if counter_Re == 3: counter_Re, counter_Im = 6, 9
    axs[counter_Re].plot(scotty_3d_data["tau_values"], scotty_3d_data[f"{element_key}_Re"], color='blue', linestyle='-', linewidth=2, zorder=1, label=f"3d Scotty {element}_Re value") # numerical
    axs[counter_Re].plot(scotty_2d_data["tau_values"], scotty_2d_data[f"{element_key}_Re"], color='red', linestyle='--', linewidth=2, zorder=1, label=f"2d Scotty {element}_Re value") # numerical
    axs[counter_Re].set_title(f"{element}_Re vs tau")
    axs[counter_Re].grid(True, which="both")
    axs[counter_Re].legend(loc="best")
    counter_Re += 1
    axs[counter_Im].plot(scotty_3d_data["tau_values"], scotty_3d_data[f"{element_key}_Im"], color='blue', linestyle='-', linewidth=2, zorder=1, label=f"3d Scotty {element}_Im value") # numerical
    axs[counter_Im].plot(scotty_2d_data["tau_values"], scotty_2d_data[f"{element_key}_Im"], color='red', linestyle='--', linewidth=2, zorder=1, label=f"2d Scotty {element}_Im value") # numerical
    axs[counter_Im].set_title(f"{element}_Im vs tau")
    axs[counter_Im].grid(True, which="both")
    axs[counter_Im].legend(loc="best")
    counter_Im += 1
plt.show()



""" FIRST DERIVATIVES """
fig, axs = plt.subplots(2, 3, figsize=(20, 10))
axs = axs.flatten()
plt.tight_layout()
counter = 0
for element in ["dH_dX", "dH_dY", "dH_dZ", "dH_dKx", "dH_dKy", "dH_dKz"]:
    axs[counter].plot(scotty_3d_data["tau_values"], scotty_3d_data[f"{element}"], color='blue', linestyle='-', linewidth=2, zorder=1, label=f"3d Scotty {element} value") # numerical
    axs[counter].plot(scotty_2d_data["tau_values"], scotty_2d_data[f"{element}"], color='red', linestyle='--', linewidth=2, zorder=1, label=f"2d Scotty {element} value") # numerical
    axs[counter].set_title(f"{element} vs tau")
    axs[counter].grid(True, which="both")
    axs[counter].legend(loc="best")
    counter += 1
plt.show()